<a href="https://colab.research.google.com/github/DivyaShreeK-dev/sdc/blob/main/medical_using_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests transformers bs4 sentence-transformers langchain faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Function to search PubMed and retrieve relevant articles
def search_pubmed(query, max_results=5):
    # PubMed API endpoint
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': max_results,
        'retmode': 'xml',
        'sort': 'relevance',
    }

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception("Error searching PubMed")

    # Parse the response and extract article IDs
    soup = BeautifulSoup(response.content, 'xml')
    article_ids = [id_tag.text for id_tag in soup.find_all('Id')]
    return article_ids

# 2. Function to fetch article details from PubMed
def fetch_article_details(article_ids):
    # PubMed article details API endpoint
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    ids = ','.join(article_ids)
    params = {
        'db': 'pubmed',
        'id': ids,
        'retmode': 'xml',
    }

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception("Error fetching article details from PubMed")

    # Parse the response and extract article titles and abstracts
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for article in soup.find_all('PubmedArticle'):
        title = article.find('ArticleTitle').text if article.find('ArticleTitle') else 'No title'
        abstract = article.find('AbstractText').text if article.find('AbstractText') else 'No abstract available'
        articles.append(f"Title: {title}\nAbstract: {abstract}")
    return articles

# 3. Function to create a medical Q&A chatbot
def medical_qa_chatbot(query):
    # Search PubMed for relevant articles
    print(f"🔍 Searching PubMed for: {query}")
    article_ids = search_pubmed(query, max_results=5)

    if not article_ids:
        return "Sorry, I couldn't find any relevant articles on PubMed."

    # Fetch article details (titles and abstracts)
    articles = fetch_article_details(article_ids)

    # Combine articles into one document for summarization
    documents = "\n\n".join(articles)

    # Use SentenceTransformer to create embeddings
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode([documents])

    # Create FAISS index to store and retrieve the embeddings
    index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance (Euclidean distance)
    index.add(np.array([embeddings[0]]).astype(np.float32))

    # Use HuggingFace model for question answering
    qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

    # Retrieve relevant documents from FAISS index
    retrieved_text = documents  # In this case, we're directly using the full text

    # Use the question answering model to answer based on the retrieved text
    answer = qa_model(question=query, context=retrieved_text)

    return answer['answer']

# 4. Example usage
query = input("Enter your medical question: ")
response = medical_qa_chatbot(query)
print("\n🔑 Answer:", response)


Enter your medical question: symptoms of diabetes
🔍 Searching PubMed for: symptoms of diabetes


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu



🔑 Answer: excretion of large amounts of hypotonic urine
